# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
! ls ./event_data

2018-11-01-events.csv  2018-11-11-events.csv  2018-11-21-events.csv
2018-11-02-events.csv  2018-11-12-events.csv  2018-11-22-events.csv
2018-11-03-events.csv  2018-11-13-events.csv  2018-11-23-events.csv
2018-11-04-events.csv  2018-11-14-events.csv  2018-11-24-events.csv
2018-11-05-events.csv  2018-11-15-events.csv  2018-11-25-events.csv
2018-11-06-events.csv  2018-11-16-events.csv  2018-11-26-events.csv
2018-11-07-events.csv  2018-11-17-events.csv  2018-11-27-events.csv
2018-11-08-events.csv  2018-11-18-events.csv  2018-11-28-events.csv
2018-11-09-events.csv  2018-11-19-events.csv  2018-11-29-events.csv
2018-11-10-events.csv  2018-11-20-events.csv  2018-11-30-events.csv


In [6]:
songs = pd.read_csv('event_datafile_new.csv')

In [7]:
songs.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [14]:
songs.head(10)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38
5,Carrie Underwood,Gianna,F,2,Jones,195.70893,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Look At Me,38
6,No Te Va Gustar,Kaylee,F,0,Summers,246.43873,free,"Phoenix-Mesa-Scottsdale, AZ",833,Ya EntendÃÂ­,8
7,The Velvet Underground / Nico,Kaylee,F,1,Summers,360.09751,free,"Phoenix-Mesa-Scottsdale, AZ",833,All Tomorrow's Parties,8
8,Snoop Dogg / Jamie Foxx,Kaylee,F,2,Summers,179.35628,free,"Phoenix-Mesa-Scottsdale, AZ",833,Psst!,8
9,Dixie Chicks,Kaylee,F,3,Summers,201.01179,free,"Phoenix-Mesa-Scottsdale, AZ",833,There's Your Trouble,8


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [11]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [12]:
# TO-DO: CREATE KEYSPACE 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS project 
                    WITH REPLICATION =
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

except Exception as e:
    print(e)

#### Set Keyspace

In [13]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)


In [37]:
query = "CREATE TABLE IF NOT EXISTS event_data"
query = query + "(artist text, firstName text, gender text , itemInSession int , lastName text, length float, level text, \
                    location text, sessionId int, song text, userId text, PRIMARY KEY(artist, song))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [28]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length FROM event_data WHERE sessionId = 338 AND itemInSession = 4"


                    

In [38]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO event_data(artist, firstName, gender, itemInSession, lastName, length,level, location, sessionId, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]),line[6], line[7], int(line[8]), line[9],line[10]))

#### Do a SELECT to verify that the data have been inserted into each table

In [40]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM event_data WHERE sessionId = 338 AND itemInSession = 4 ALLOW FILTERING;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [44]:
query = "CREATE TABLE IF NOT EXISTS event_data1"
query = query + "(artist text, firstName text, gender text , itemInSession int , lastName text, length float, level text, \
                    location text, sessionId int, song text, userId text, PRIMARY KEY(artist, song, sessionid, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO event_data1(artist, firstName, gender, itemInSession, lastName, length,level, location, sessionId, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]),line[6], line[7], int(line[8]), line[9],line[10]))

In [51]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query1 = "SELECT artist, song FROM event_data1 WHERE userId = '10' AND sessionId = 182 ALLOW FILTERING;"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    

for row in rows:
    print (row.artist, row.song)                    

Sebastien Tellier Kilometer
Three Drives Greece 2000
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
Down To The Bone Keep On Keepin' On


In [61]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query2 = "SELECT firstName, lastName FROM event_data1 WHERE song = 'All Hands Against His Own' ALLOW FILTERING;"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    

for row in rows:
    print (row.firstname, row.lastname)   
                    

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [62]:
query = "drop table event_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table event_data1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [63]:
session.shutdown()
cluster.shutdown()